In [2]:
import ccxt
import price_data as price
import chart
import time
import schedule
import numpy as np
from datetime import datetime
from ftx_client import FtxClient

ftx_ccxt = ccxt.ftx({
    'apiKey': 'mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',
    'secret': 'oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B',
    'enableRateLimit': True,
})

# ftx_ccxt.headers = {'FTX-SUBACCOUNT':'MeanReversion'}

main=FtxClient(api_key='mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',api_secret='oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B')
Savings=FtxClient(api_key='mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',api_secret='oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B',subaccount_name='Savings')
MeanReversion=FtxClient(api_key='mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',api_secret='oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B',subaccount_name='MeanReversion')

In [1]:
import price_data as price
import chart
import pandas as pd
import numpy as np
import ccxt
from datetime import datetime
from users import users
from ftx_client import FtxClient

ftx_ccxt = ccxt.ftx({
    'apiKey': 'mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',
    'secret': 'oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B',
    'enableRateLimit': True
})
ftx_ccxt.password = 'Ev443978??'
cex= ccxt.cex({
    'uid' : 'up109520414',
    'apiKey': '1X2uEcPlvBCe4CcMtzWKguG1SDI',
    'secret': '8JY4fDg6hRz0DTolZHz77XPdC1o',
    'enableRateLimit': True,
    })

ftx_ccxt.headers = {'FTX-SUBACCOUNT':'Savings'}

def get_limit(symbol, markets):
    return float(next(filter(lambda x:x['symbol']==symbol, markets))['limits']['amount']['min'])


In [3]:
ftx_ccxt.fetch_partial_balance('USD')['total']

3536.88886287

In [4]:
# position=main.get_position('ETH-PERP',True)
# print(main.get_conditional_orders())
# balance=float(next(filter(lambda x:x['coin']=='USD', ShortTerm.get_balances()))['total'])
# PnL=float(position['recentPnl'])
# percentage_profit=(PnL/balance)*100
# tp_amount=np.log(percentage_profit)/10
# position_size=float(position['size'])

In [5]:
noDecimals=np.absolute(np.log10(next(filter(lambda x:x['symbol']=='MATIC/USD', ftx_ccxt.fetch_markets()))['precision']['amount']))

In [9]:
rob = ccxt.ftx(users['rob']['exchanges']['ftx']['api'])

TypeError: 'module' object is not subscriptable

In [7]:
def transfer_ftx_to_cex(ftx_dict: dict,cex: dict,usd_amount: float,headers: dict=None, blockchain = 'SOL'):
    ftx = ccxt.ftx(ftx_dict)
    ftx.password=ftx_dict['password']
    cex = ccxt.cex(cex)
    if headers != None:
        ftx.headers = headers
    symbol=blockchain+'/'+'USD'
    current_price=price.get_current_price(symbol)
    amount = usd_amount/current_price
    noDecimals=int(np.absolute(np.log10(next(filter(lambda x:x['symbol']==symbol, ftx.fetch_markets()))['precision']['amount'])))
    amount_to_buy = round(amount,noDecimals)
    deposit_address = cex.fetchDepositAddress(blockchain)
    # ftx.createMarketBuyOrder(symbol='SOL/USD', amount=amount_to_buy)
    ftx.withdraw(code=blockchain, amount=amount_to_buy, address=deposit_address['address'])
    

my_ftx = {
    'apiKey': 'mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',
    'secret': 'oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B',
    'enableRateLimit': True,
    'password': 'Ev443978??'
}

my_cex = {
    'uid' : 'up109520414',
    'apiKey': '1X2uEcPlvBCe4CcMtzWKguG1SDI',
    'secret': '8JY4fDg6hRz0DTolZHz77XPdC1o',
    'enableRateLimit': True,
}

my_savings = {'FTX-SUBACCOUNT':'Savings'}
# transfer_ftx_to_cex(my_ftx,my_cex,50, my_savings)

users_=users
for key in users_:
    print(users_[key])

{'daily_buy_amount': 10, 'exchanges': {'ftx': {'symbols': ['BTC/USD', 'ETH/USD', 'SOL/USD', 'MATIC/USD'], 'api': {'apiKey': 'mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn', 'secret': 'oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B', 'enableRateLimit': True}, 'header': {'FTX-SUBACCOUNT': 'Savings'}}, 'cex': {'symbols': ['ADA/USD'], 'api': {'uid': 'up109520414', 'apiKey': '1X2uEcPlvBCe4CcMtzWKguG1SDI', 'secret': '8JY4fDg6hRz0DTolZHz77XPdC1o', 'enableRateLimit': True}}}}
{'daily_buy_amount': 10, 'exchanges': {'ftx': {'symbols': ['BTC/USD', 'ETH/USD', 'SOL/USD', 'MATIC/USD'], 'api': {'apiKey': 'mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn', 'secret': 'oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B', 'enableRateLimit': True}}}}
